<a href="https://colab.research.google.com/github/shubhamgupta1017/Multi-Class-Prediction-of-Cirrhosis-Outcomes/blob/main/Multi_Class_Prediction_of_Cirrhosis_Outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle compedition

```
Multi-Class Prediction of Cirrhosis Outcomes
```



importing library

In [629]:
import numpy as np
import pandas as pd

In [630]:
df=pd.read_csv("/content/train (1).csv")
X_sub=pd.read_csv("/content/test (1).csv")

In [631]:
X=df.iloc[:,1:-1].values
y=df.iloc[:,-1].values
X_sub=X_sub.iloc[:,1:].values

preproccesing data

In [632]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
to_transform=[1,3,4,5,6,7]
for i in to_transform:
  X_sub[:,i] = le.fit_transform(X_sub[:, i])
  X[:,i] = le.fit_transform(X[:, i])

In [633]:
X = X.astype('float32')
X_sub=X_sub.astype('float32')

In [634]:
X[0]

array([9.9900e+02, 0.0000e+00, 2.1532e+04, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 2.3000e+00, 3.1600e+02,
       3.3500e+00, 1.7200e+02, 1.6010e+03, 1.7980e+02, 6.3000e+01,
       3.9400e+02, 9.7000e+00, 3.0000e+00], dtype=float32)

In [635]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X[:,1:])
X_sub = sc.transform(X_sub[:,1:])

In [636]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

PCA

In [637]:
from sklearn.decomposition import PCA
n_components = 16
pca = PCA(n_components=n_components)
X = pca.fit_transform(X)
X_sub = pca.transform(X_sub)

Model

In [638]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split

In [639]:
# @title fine tunning hyper prameter
# xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=3)
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_log_loss', cv=3)
# grid_search.fit(X, y)
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X)


# @title fine tunning hyperparamter
# param_grid = {
#     'max_depth': [2],
#     'learning_rate': [0.1],
#      'n_estimators': [143],
#     'subsample': [0.74],
#     'colsample_bytree': [ 1.0],
# }


In [640]:
model = xgb.XGBClassifier(
    max_depth=2,
    learning_rate=0.1,
    n_estimators=143,
    subsample=0.74,
    colsample_bytree=1.0,
    objective='multi:softprob',  # Assuming a multi-class problem
    num_class=3 # Specify the number of classes in your problem
)

model.fit(X, y)
y_pred = model.predict(X)
y_probabilities = model.predict_proba(X)
print(y_probabilities[:5])

[[0.41304427 0.03572679 0.55122894]
 [0.8470031  0.0238042  0.12919264]
 [0.1020425  0.02063897 0.8773185 ]
 [0.9163221  0.01215103 0.07152684]
 [0.68503296 0.05528663 0.25968042]]


In [641]:
logloss = log_loss(y, y_probabilities)
print(f'Log Loss: {logloss:.4f}')#no need of confusion matrix

Log Loss: 0.4724


In [642]:
y_predic = model.predict_proba(X_sub)
print(y_predic)

[[0.7329399  0.03903151 0.2280286 ]
 [0.86991197 0.05696763 0.07312037]
 [0.187244   0.03683708 0.7759189 ]
 ...
 [0.7805555  0.0414056  0.17803895]
 [0.94985336 0.02680028 0.02334633]
 [0.50649345 0.01463132 0.47887528]]


final exporting of result

In [643]:
sub=pd.read_csv("/content/test (1).csv")
output = np.column_stack((sub.iloc[:, 0], y_predic))
output[:,0]=output[:,0].astype(int)
output = pd.DataFrame(output, columns=['id', 'Status_C', 'Status_CL', 'Status_D'])
print(output)

           id  Status_C  Status_CL  Status_D
0      7905.0  0.732940   0.039032  0.228029
1      7906.0  0.869912   0.056968  0.073120
2      7907.0  0.187244   0.036837  0.775919
3      7908.0  0.938368   0.010668  0.050964
4      7909.0  0.667229   0.012963  0.319808
...       ...       ...        ...       ...
5266  13171.0  0.894828   0.030746  0.074426
5267  13172.0  0.941649   0.007968  0.050384
5268  13173.0  0.780555   0.041406  0.178039
5269  13174.0  0.949853   0.026800  0.023346
5270  13175.0  0.506493   0.014631  0.478875

[5271 rows x 4 columns]


In [644]:
output.to_csv('/content/output_file.csv', index=False)
from google.colab import files

files.download('/content/output_file.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>